<h3>Import nessesary libraries</h3>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.exceptions import DataConversionWarning
import warnings
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)
np.random.seed(42)

<h3>Data preprocessing</h3>

In [29]:
df_train = pd.read_excel('Task_Python.xlsx', sheet_name='Train')
df_test = pd.read_excel('Task_Python.xlsx', sheet_name='Test')

In [47]:
X_train_pre = df_train.drop(['y'], axis=1)
y_train_pre = np.where(df_train.y == 'yes', 1, 0)
X_test_pre = df_test
columns = X_train_pre.columns
# We will use list of categorical variables later
categorical_features = []
is_categorical = X_train_pre.dtypes == 'object'
for col in X_train_pre.columns:
    if is_categorical[col]: categorical_features.append(col)

In [48]:
X_train = X_train_pre.reset_index(drop=True)
y_train = y_train_pre
# Encoding for categorical variables, as most models don't work with strings
X_train = pd.get_dummies(X_train, columns=categorical_features, drop_first=True)
# for feature in categorical_features:
#     le = LabelEncoder()
#     X_train[feature]  = le.fit_transform(X_train[feature])
# one = OneHotEncoder(categorical_features=[X_train.columns.get_loc(i) for i in categorical_features])
# X_train = one.fit_transform(X_train).toarray()
# Standartization, as most models are sensetive to magnitude of variables
Scaler = StandardScaler(with_mean=False)
# columns = X_train.columns
X_train = Scaler.fit_transform(X_train)
# Our data is imbalanced (1:9). So, we use SMOTE to synthesize more class 1 observations
smote = SMOTE()
X_train, y_train = smote.fit_sample(X_train, y_train)
# X_train = pd.DataFrame(X_train, columns = columns)

<h1>Modeling</h1>

In [49]:
model_list = [
            ("Logistic_Regression", LogisticRegression()),
            ("SVC", SVC()),
            ("LDA", LinearDiscriminantAnalysis()),
            ("QDA", QuadraticDiscriminantAnalysis()),
            ("RF", RandomForestClassifier()),
            ("GBC", GradientBoostingClassifier()),
            ("XGB", XGBClassifier()),
            ("LGBM", LGBMClassifier())
        ]  
    
scoring = 'f1'
n_folds = 5
results, names  = [], [] 

for name, model  in model_list:
    kfold = StratifiedKFold(n_splits=n_folds, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv= kfold,
                                 scoring=scoring, n_jobs=-1)    
    names.append(name)
    results.append(cv_results)    
    print(f'{name}: {cv_results.mean()} (+/- {cv_results.std()})')
    
results = pd.DataFrame(np.array(results).T, columns = names)

Logistic_Regression: 0.8947402815778542 (+/- 0.010602669525754749)
SVC: 0.9490280004599647 (+/- 0.0076458285694324364)
LDA: 0.8857006685167738 (+/- 0.009350425909110331)
QDA: 0.7027055312386444 (+/- 0.044288040299362)
RF: 0.9518037213153834 (+/- 0.0051253018072496)
GBC: 0.941323428386337 (+/- 0.005655493033103703)
XGB: 0.9420774122464783 (+/- 0.007277938673366541)
LGBM: 0.9553533492004276 (+/- 0.0067944646931446875)


<h5>Cat Boosting</h5><br>This model has its own specific requirements (categotical variables can be left alone and passed to a model without encoding)

In [6]:
X_train = X_train_pre.reset_index(drop=True)
y_train = y_train_pre
for col in X_train.columns:
    if col not in categorical_features:
        Scaler = StandardScaler()
        X_train[col] = Scaler.fit_transform(X_train[col].values.reshape(-1,1))

smotenc = SMOTENC(categorical_features=[X_train.columns.get_loc(i) for i in categorical_features])
X_train, y_train = smotenc.fit_sample(X_train, y_train)
X_train = pd.DataFrame(X_train, columns = columns)

In [7]:
model = CatBoostClassifier(cat_features=categorical_features)

scoring = 'f1'
n_folds = 5

kfold = StratifiedKFold(n_splits=n_folds, shuffle=True)
cv_results = cross_val_score(model, X_train, y_train, cv= kfold,
                             scoring=scoring, n_jobs=-1)    

print(f'Cat_Boosting: {cv_results.mean()} (+/- {cv_results.std()})')

Cat_Boosting: 0.9415684137438074 (+/- 0.005140694316555464)


<h3>Regularization of the best model</h3><h5>Light gradient boosting machine hyperparameters tuning</h5>

In [50]:
model = LGBMClassifier()

param_grid_et =  {
        'n_estimators': [100, 300, 500],
        'learning_rate': [0.01, 0.05, 0.1],
        'max_depth': [5, 10, 15],
        'num_leaves': [200, 500, 900]
               }

grid_rf = GridSearchCV(estimator=model,
                               param_grid= param_grid_et,
                               scoring= 'f1',
                               cv = StratifiedKFold(n_splits=3, shuffle=True),
                               n_jobs = 4,
                               verbose = 10)
# Fit grid
grid_rf.fit(X_train, y_train)

# Best score and best parameters
print('-------Best score----------')
print(grid_rf.best_score_ )
print('-------Best params----------')
print(grid_rf.best_params_)

Fitting 3 folds for each of 81 candidates, totalling 243 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    5.5s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    9.8s
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:   18.8s
C:\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:   26.8s
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   45.2s
[Parallel(n_jobs=4)]: Done  53 tasks      | elapsed:  1.3min
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:  1.6min
[Parallel(n_jobs=4)]: Done  77 tasks      | elapsed:  2.4min
[Parallel(n_jobs=4)]: Done  90 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 10

-------Best score----------
0.9542962660072529
-------Best params----------
{'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 300, 'num_leaves': 200}


<h1>Prediction</h1><br>

In [15]:
X_test = X_test_pre.reset_index(drop=True)
for feature in categorical_features:
    le = LabelEncoder()
    X_test[feature]  = le.fit_transform(X_test[feature])
X_test = pd.DataFrame(Scaler.transform(X_test), columns = columns)

In [17]:
# Our final model
model = LGBMClassifier(
    learning_rate=0.1,
    max_depth=15,
    n_estimators=300,
    num_leaves=200
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)